In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import jieba
import pickle
import re

In [3]:
#将语料库转为dataframe格式

# text_list = glob('./THUCNews\\*\\*.txt')
# text_label = list(map(lambda x:x.split('\\')[-2],text_list))
# df = pd.DataFrame()
# values = []

# for index in range(len(text_list)):
#     myfile=open(text_list[index],encoding='utf_8')
#     a=myfile.read()
#     values.append([a,text_label[index]])
# df = df.append(values)
# df.columns=['content','label']
# df.to_pickle('train.pkl')

In [4]:
# 对某些过多的类别进行降采样

# %%time
# train = pd.read_pickle('train.pkl')
# down_label = ['科技','股票','体育','娱乐','时政']
# for i in down_label:
#     drop_index = train[train.label==i].index
#     drop_index = drop_index[:len(drop_index)//2]
#     train = train.drop(drop_index)
# train.to_pickle('down_sample.pkl')

In [5]:
# #tf_idf向量化 max_features=1000
# train = pd.read_pickle('down_sample.pkl')
# train = train.reset_index(drop=True)
# sencents = train.content.values
# content = []
# for i in tqdm(sencents):
#     xx = jieba.lcut(re.compile(u"[^\u4e00-\u9fa5]+").sub('', i))  
#     content.append(",".join(xx))
# vectorizer = TfidfVectorizer(min_df=15,max_df=0.85,max_features=1000) 
# x = vectorizer.fit_transform(content)
# word_tfidf = pd.DataFrame(x.toarray(),columns=vectorizer.get_feature_names())
# word_tfidf.to_pickle('tfidf.pkl')

In [6]:
# pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))
# vectorizer = pickle.load(open("vectorizer.pickle", "rb"))

In [18]:
#tf_idf中添加特征以及标签

# train = pd.read_pickle('./data/down_sample.pkl')
# train = train.reset_index(drop=True)
# labels = train.label

# tfidf = pd.read_pickle('./data/tfidf.pkl')
# tfidf['len'] = list(map(lambda x:len(x),train.content))
# tfidf['label'] = labels
# tfidf.to_pickle('tfidf.pkl')

In [7]:
df = pd.read_pickle('./data/tfidf.pkl')
data = df.drop(['label','len'],axis=1)#长度应该还是不适合做特征
target = df.label
data.columns = range(data.shape[1])
del df

In [8]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y = train_test_split(data,target,test_size=0.2,random_state=10100)

In [9]:
from lightgbm import LGBMClassifier

model = LGBMClassifier(n_estimators=100000)
model.fit(train_x,train_y,eval_set=[(test_x,test_y)],eval_metric='error',early_stopping_rounds=20,verbose=True)

[1]	valid_0's multi_error: 0.216925	valid_0's multi_logloss: 1.67618
Training until validation scores don't improve for 20 rounds
[2]	valid_0's multi_error: 0.177806	valid_0's multi_logloss: 1.40838
[3]	valid_0's multi_error: 0.161722	valid_0's multi_logloss: 1.22379
[4]	valid_0's multi_error: 0.147165	valid_0's multi_logloss: 1.08487
[5]	valid_0's multi_error: 0.140017	valid_0's multi_logloss: 0.973306
[6]	valid_0's multi_error: 0.135296	valid_0's multi_logloss: 0.884443
[7]	valid_0's multi_error: 0.132167	valid_0's multi_logloss: 0.81255
[8]	valid_0's multi_error: 0.130097	valid_0's multi_logloss: 0.751614
[9]	valid_0's multi_error: 0.127896	valid_0's multi_logloss: 0.699518
[10]	valid_0's multi_error: 0.125872	valid_0's multi_logloss: 0.655668
[11]	valid_0's multi_error: 0.123943	valid_0's multi_logloss: 0.617614
[12]	valid_0's multi_error: 0.122481	valid_0's multi_logloss: 0.586771
[13]	valid_0's multi_error: 0.120514	valid_0's multi_logloss: 0.556969
[14]	valid_0's multi_error: 0.

LGBMClassifier(n_estimators=100000)

In [22]:
# pickle.dump(model, open("lgb_model.pickle", "wb"))

In [39]:
vectorizer = pickle.load(open("vectorizer.pickle", "rb"))
model = pickle.load(open("lgb_model.pickle", "rb"))

In [40]:
test_x = vectorizer.transform([s]).toarray()

In [41]:
model.predict(test_x)

array(['科技'], dtype=object)